In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = 1200
DEFAULT_END = "_end"
DEFAULT_PASS =" "
DEFAULT_UNK =''

разбивка текста и создание словаря

In [15]:
from Tokenizer import Tokenizer
text = open(r"D:\Projects\RnnTextGen\text.txt",encoding="utf-8").read().lower()
words = text.split()
questions = text.splitlines()[::2]
answers = text.splitlines()[1::2]
tokenizer = Tokenizer(VOCAB_SIZE)
tokenizer.fit([text],DEFAULT_END,DEFAULT_UNK)

создание модели 
предложение -> hidden
последние слово + hidden -> слово(1)...слово(n)

In [7]:
class RnnTextGen(nn.Module):

    def __init__(self,voc_size,inp_size,hid_size,n_layers,dropout=0.2) -> None:
        super(RnnTextGen,self).__init__()
        self.voc_size = voc_size
        self.n_layers = n_layers
        self.hidden_size=hid_size
        self.Encoder = nn.Embedding(voc_size,inp_size)
        self.lstm = nn.LSTM(inp_size,hid_size,n_layers)
        self.dropout = nn.Dropout(dropout)
        self.l1 = nn.Linear(hid_size,voc_size)
        
    def forward(self,x,hidden=None):
        x = self.Encoder(x)
        x,hidden = self.lstm(x)
        x = self.dropout(x)
        x = self.l1(x)
        return x,hidden
    
    def init_hidden(self,batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [8]:
model=RnnTextGen(VOCAB_SIZE,1000,500,2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)


In [9]:
def evaluate(model:RnnTextGen,text:str,prediction_lim:int=15):
    text_idx = torch.LongTensor(list(tokenizer.tokenize(text))).to(device)
    hidden = model.init_hidden()
    inp = text_idx
    predicted_text=""
    for i in range(prediction_lim):
        next_w , hidden = model(inp.view(-1,1).to(device),hidden)
        inp = torch.cat([inp,next_w[-1].argmax().view(-1)])
        word = tokenizer.rw_tokens[int(next_w[-1].argmax())]
        if next_w[-1].argmax() == torch.LongTensor([0]).to(device):
            break
        predicted_text +=word
    return predicted_text

In [10]:
def get_batch(questions:list,answers:list):
    for question,answer in zip(questions,answers):
        question_idx = list(tokenizer.tokenize(question))
        target = list(tokenizer.tokenize(answer))+[0]
        test = question_idx+target[:-1]

        target =torch.LongTensor(target).to(device)
        test = torch.LongTensor(test).to(device)
        yield target,test

In [11]:
def train(epoches:int,model:RnnTextGen,batch_size:int)->None:
    """epoches - number of epoches through all dataset
    model - model required to teach
    batch_size - n/a"""
    loss_avg =[]
    for epoch in range(epoches):
        for target,train in get_batch(questions,answers):
            model.train()

            hidden = model.init_hidden(batch_size)

            output,hidden = model(train,hidden)
            target_len = len(target)
            loss = criterion(output[-target_len:],target)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_avg.append(loss.item())
            if len(loss_avg) >= 50:
                mean_loss = np.mean(loss_avg)
                print(f'Loss: {mean_loss}')
                scheduler.step(mean_loss)
                model.eval()
                question = random.choice(questions)
                answer = evaluate(model,question)
                print(f"Question: {question} \n Answer: {answer}")
                loss_avg = []

обучение модели

In [16]:
train(10,model,1)

KeyError: 35

In [ ]:
quest = input()
evaluate(model,quest,35)

'привет '

In [ ]:
torch.save(model,"data.pkl")

NameError: name 'torch' is not defined

In [ ]:
model = torch.load("data.pkl").to(device)